# Trajectory Analysis using 10x Single Cell Gene Expression Data
More details on this analysis can be found in the 10x Genomics Analysis Guides tutorial, located here: https://www.10xgenomics.com/resources/analysis-guides/trajectory-analysis-using-10x-Genomics-single-cell-gene-expression-data

June 28, 2022

This will install the required Python library packages needed for this tutorial:

In [ ]:
!pip install numpy==1.19.2 pandas==1.1.5 matplotlib===3.3.4 scanpy==1.7.2 igraph==0.9.8 scvelo==0.2.4 loompy==3.0.6 anndata==0.7.8

This will create a new directory called "input-files", then download and extract several input data files needed for this tutorial, then display the list of files now available.

In [ ]:
!mkdir input-files
!curl -o input-files/filtered_feature_bc_matrix.tar.gz https://cf.10xgenomics.com/supp/cell-exp/neutrophils/filtered_feature_bc_matrix.tar.gz
!curl -o input-files/WB_Lysis_3p_Introns_8kCells.loom https://cf.10xgenomics.com/supp/cell-exp/neutrophils/WB_Lysis_3p_Introns_8kCells.loom
!curl -o input-files/3p-Neutrophils-clusters.csv https://cf.10xgenomics.com/supp/cell-exp/neutrophils/3p-Neutrophils-clusters.csv
!curl -o input-files/3p-Neutrophils-UMAP-Projection.csv https://cf.10xgenomics.com/supp/cell-exp/neutrophils/3p-Neutrophils-UMAP-Projection.
!tar -xvzf input-files/filtered_feature_bc_matrix.tar.gz -C input-files/
!ls -lah input-files

In [3]:
# First, import required packages in the current session.

import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import scanpy as sc
import igraph
import scvelo as scv
import loompy as lmp
import anndata

import warnings
warnings.filterwarnings('ignore')

This chunk of code below is used to customize colors for the cluster and plot sizes

In [4]:
# Customize parameters for plots (Size, Color, etc)
scv.set_figure_params(style="scvelo")
pl.rcParams["figure.figsize"] = (10,10)
Colorss=["#E41A1C","#377EB8","#4DAF4A","#984EA3","#FF7F00","#FFFF33","#A65628","#F781BF"]

Import the GEX matrix along with the clusters and UMAP embeddings from Loupe.



In [5]:
# Define Path to cellranger output
Path10x='./input-files/filtered_feature_bc_matrix/'

# Read cellranger output
Neutro3p = sc.read_10x_mtx(Path10x,var_names='gene_symbols',cache=True)

Neutro3p

AnnData object with n_obs × n_vars = 8000 × 36601 
    var: 'gene_ids', 'feature_types'

In [6]:
# Read Clusters exported from Loupe Browser 
Clusters_Loupe = pd.read_csv("./input-files/3p-Neutrophils-clusters.csv", delimiter=',',index_col=0)

# Create list with Neutrophil Barcodes
Neutrophils_BCs = Clusters_Loupe.index

# Read UMAP exported from Loupe Browser 
UMAP_Loupe = pd.read_csv("./input-files/3p-Neutrophils-UMAP-Projection.csv", delimiter=',',index_col=0)

# Select Neutrophil Barcodes
UMAP_Loupe = UMAP_Loupe.loc[Neutrophils_BCs,]

# Tansform to Numpy
UMAP_Loupe = UMAP_Loupe.to_numpy()

# Filter Cells to only Neutrophils
Neutro3p = Neutro3p[Neutrophils_BCs]

# Add Clusters from Loupe to object
Neutro3p.obs['Loupe'] = Clusters_Loupe

# Add UMAP from Loupe to object
Neutro3p.obsm["X_umap"] = UMAP_Loupe

Neutro3p

KeyError: "None of [Index(['AAACCCAGTTGTGCAT-1', 'AAACGAACAATTGCGT-1', 'AAACGCTTCGAGTCCG-1',\n       'AAAGAACAGCGACTAG-1', 'AAAGAACAGTCGAGGT-1', 'AAAGAACCACTGCACG-1',\n       'AAAGAACCAGTGCGCT-1', 'AAAGAACTCAGCTCTC-1', 'AAAGGATAGGAGCTGT-1',\n       'AAAGGATCACCCAAGC-1',\n       ...\n       'TTTGGAGTCGCGTTTC-1', 'TTTGGAGTCGTTCTAT-1', 'TTTGGTTAGTAGTCCT-1',\n       'TTTGGTTCAATAGTAG-1', 'TTTGGTTTCGAACGCC-1', 'TTTGTTGAGTAAGCAT-1',\n       'TTTGTTGCAACCAACT-1', 'TTTGTTGTCGAGTACT-1', 'TTTGTTGTCGCTACAA-1',\n       'TTTGTTGTCTTCGCTG-1'],\n      dtype='object', name='Barcode', length=3343)] are in the [index]"

You might get this warning below, but nothing to worry about.
> Trying to set attribute `.obs` of view, copying.

Next, read velocyto output and merge

In [ ]:
# Read velocyto output
VelNeutro3p = scv.read('./input-files/WB_Lysis_3p_Introns_8kCells.loom', cache=True)

# Merge velocyto and cellranger outputs
Neutro3p = scv.utils.merge(Neutro3p, VelNeutro3p)

Neutro3p

You might get this warning, but no need to worry:
> Variable names are not unique. To make them unique, call `.var_names_make_unique`.

Next, process dataset and obtain latent time values for each cell


In [ ]:
# Standard scvelo processing to run Dynamical Mode
scv.pp.filter_and_normalize(Neutro3p, min_shared_counts=30, n_top_genes=2000)
scv.pp.moments(Neutro3p, n_pcs=30, n_neighbors=30)

scv.tl.recover_dynamics(Neutro3p)
scv.tl.velocity(Neutro3p, mode='dynamical')
scv.tl.velocity_graph(Neutro3p)
scv.tl.recover_latent_time(Neutro3p)

Neutro3p

In [ ]:
# Visualize the results

scv.pl.velocity_embedding_stream(Neutro3p,basis="umap",color="Loupe",title='Neutrophils',fontsize=20,legend_fontsize=20,min_mass=2,palette=Colorss,save='scVelo-umap-cluster.png')
scv.pl.velocity_embedding_stream(Neutro3p,basis="umap",color="latent_time",title='Neutrophils',fontsize=20,legend_fontsize=20,min_mass=2,color_map="plasma",save='scVelo-umap-latent_time.png')

Genes=["RETN","LTF","CAMP","ACTB","GCA","LCN2",
         "S100A8","MYL6","S100A9","FCGR3B","S100A11","FTH1","IFIT1",
         "IFITM3","IFIT3","ISG15","IFIT2","RPS9","NEAT1","MALAT1","NFKBIA","CXCL8"]

scv.pl.heatmap(Neutro3p, var_names=Genes, sortby='latent_time', col_color='Loupe', n_convolve=100,figsize=(16,8),yticklabels=True,sort=True,colorbar=True,show=True,layer="count", save='scVelo-heatmap-latent_time.png')
sc.pl.violin(Neutro3p, keys='latent_time',groupby="Loupe",order=["Cluster 4","Cluster 1","Cluster 5","Cluster 6","Cluster 7","Cluster 3","Cluster 2"], save='scVelo-violin-latent_time.png')
